In [63]:
import pandas as pd
from konlpy.tag import Okt
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.metrics.pairwise import cosine_similarity
import numpy as np

# 데이터 로드
data = pd.read_csv('20171104_test.csv', index_col=0)

# 형태소 분석기 초기화
okt = Okt()

# 기사의 내용을 형태소로 분석
data['tokenized_content'] = data['content'].apply(lambda x: ' '.join(okt.morphs(x)))

# TF-IDF 변환기를 초기화
tfidf_vectorizer = TfidfVectorizer()

# 형태소 분석된 내용을 벡터로 변환
tfidf_matrix = tfidf_vectorizer.fit_transform(data['tokenized_content'])

# 코사인 유사도 행렬 계산
cosine_sim = cosine_similarity(tfidf_matrix)

# 행렬의 통계 값 계산
# Compute statistics for the matrix
min_value = round(np.min(cosine_sim), 4)
max_value = round(np.max(cosine_sim), 4)
mean_value = round(np.mean(cosine_sim), 4)
median_value = round(np.median(cosine_sim), 4)
std_value = round(np.std(cosine_sim), 4)

min_value, max_value, mean_value, median_value, std_value

(0.0127, 1.0, 0.2894, 0.1647, 0.3401)

In [64]:
len(data)

10

In [ ]:
# 군집화
clusters = []
visited = np.zeros(cosine_sim.shape[0], dtype=bool)
for i in range(cosine_sim.shape[0]):
    if not visited[i]:
        similar_articles = np.where(cosine_sim[i] > 0.6)[0]
        clusters.append(similar_articles)
        visited[similar_articles] = True

# 각 군집에서 대표 기사만 남기기
representative_indices = [cluster[0] for cluster in clusters if len(cluster) > 0]

# 대표 기사만 포함된 데이터프레임 생성
filtered_data = data.iloc[representative_indices].reset_index(drop=True)
print(len(filtered_data))
filtered_data

6


,time,title,content,press,link,UTC-time,UTC-date,tokenized_content
0,2017-11-04 09:56:45,잠잠한 북한…트럼프 순방기간 도발 가능성은,\r\n\t\t\t잠잠한 북한…트럼프 순방기간 도발 가능성은[앵커] 북한은 최근 5...,연합뉴스TV언론사 선정,https://n.news.naver.com/mnews/article/422/000...,2017-11-04 00:56:45,2017-11-04,\r\n\t\t\t 잠잠한 북한 … 트럼프 순방 기간 도발 가능성 은 [ 앵커 ] ...
1,2017-11-04 09:01:08,CME 비트코인 거래 개시 소식에 계좌 개설 열풍,\n\n\n\n\n[아시아경제 황준호 기자] 세계 최대 파생상품 거래소인 미국 시...,아시아경제,https://n.news.naver.com/mnews/article/277/000...,2017-11-04 00:01:08,2017-11-04,\n\n\n\n\n [ 아시아 경제 황준호 기자 ] 세계 최대 파생상품 거래소 인 ...
2,2017-11-04 15:37:27,"스파크랩, 1천억 규모 블록체인 투자 펀드 출범",\n\t\t\t 스파크랩 그룹은 블록체인과 암호화 화폐 기업 투자 펀드인 ‘스파크체...,파이낸셜뉴스,https://n.news.naver.com/mnews/article/014/000...,2017-11-04 06:37:27,2017-11-04,\n\t\t\t 스파크 랩 그룹 은 블록 체인 과 암호 화 화폐 기업 투자 펀드 ...
3,2017-11-04 19:55:00,"""현직 경찰관에 배우 동원까지""…가상화폐 범죄 기승",\r\n\t\t\t【 앵커멘트 】 하루 거래액이 2조 원을 넘어서면서 비트코인 등...,MBN,https://n.news.naver.com/mnews/article/057/000...,2017-11-04 10:55:00,2017-11-04,\r\n\t\t\t 【 앵커 멘트 】 하루 거래 액 이 2조 원 을 넘어서면서 비트...
4,2017-11-05 01:34:20,"가상화폐, 금융시장의 새로운 쓰나미",\r\n비트코인·이더리움 등 1000종류상위 10개 시가총액만 150조원빠르고 거래...,T19면 TOP,https://n.news.naver.com/mnews/article/353/000...,2017-11-04 16:34:20,2017-11-04,\r\n 비트코인 · 이 더 리움 등 1000 종류 상위 10 개 시가총액 만 15...
5,2017-11-05 06:00:00,[박수찬의 軍]대통령 공약과 따로 노는 국방부의 北 미사일 대책,\r\n\r\n\r\n\r\n\r\n천궁 지대공미사일이 2일 충남 보령 대천사격장에...,세계일보언론사 선정,https://n.news.naver.com/mnews/article/022/000...,2017-11-04 21:00:00,2017-11-04,\r\n\r\n\r\n\r\n\r\n 천궁 지 대 공 미사일 이 2일 충남 보령 대...


In [ ]:
filtered_data.to_csv('filtered_20171104_01.csv', encoding='utf-8-sig', index=False)

In [24]:
import pandas as pd
import torch
from sentence_transformers import util
from transformers import AutoTokenizer, AutoModel
import numpy as np

# 데이터 로드
data = pd.read_csv('20220119_test.csv')

# 모델과 토크나이저 로드
tokenizer = AutoTokenizer.from_pretrained('jhgan/ko-sbert-multitask')
model = AutoModel.from_pretrained('jhgan/ko-sbert-multitask')

# Mean Pooling 함수 정의
def mean_pooling(model_output, attention_mask):
    token_embeddings = model_output[0] 
    input_mask_expanded = attention_mask.unsqueeze(-1).expand(token_embeddings.size()).float()
    return torch.sum(token_embeddings * input_mask_expanded, 1) / torch.clamp(input_mask_expanded.sum(1), min=1e-9)

# 문장 토크나이즈 및 임베딩 생성
encoded_input = tokenizer(data['content'].tolist(), padding=True, truncation=True, return_tensors='pt')
with torch.no_grad():
    model_output = model(**encoded_input)
sentence_embeddings = mean_pooling(model_output, encoded_input['attention_mask'])

# 코사인 유사도 계산 함수
def cosine_similarity(embedding1, embedding2):
    return util.pytorch_cos_sim(embedding1, embedding2)

# 유사도 행렬 초기화 및 계산
similarity_matrix = []
for i in range(len(sentence_embeddings)):
    row = []
    for j in range(len(sentence_embeddings)):
        similarity = cosine_similarity(sentence_embeddings[i], sentence_embeddings[j]).item()
        row.append(similarity)
    similarity_matrix.append(row)

# 행렬의 통계 값 계산
min_value = round(np.min(similarity_matrix), 4)
max_value = round(np.max(similarity_matrix), 4)
mean_value = round(np.mean(similarity_matrix), 4)
median_value = round(np.median(similarity_matrix), 4)
std_value = round(np.std(similarity_matrix), 4)

min_value, max_value, mean_value, median_value, std_value

In [20]:
# 군집화
clusters = []
visited = np.zeros(len(data), dtype=bool)
for i in range(len(data)):
    if not visited[i]:
        similar_articles = np.where(np.array(similarity_matrix[i]) > 0.6)[0]
        clusters.append(similar_articles)
        visited[similar_articles] = True

# 각 군집에서 대표 기사만 남기기
representative_indices = [cluster[0] for cluster in clusters if len(cluster) > 0]

# 대표 기사만 포함된 데이터프레임 생성
filtered_data = data.iloc[representative_indices].reset_index(drop=True)
print(len(filtered_data))
filtered_data

18


,date,time,title,content,press,link,UTC-time,UTC-date
0,2020-12-02,2020-12-02 10:43:33,"후오비 코리아, 비트코인 1만7000달러 돌파 '앵콜' 이벤트",\r\n\r\n\r\n\r\n\r\n후오비 코리아가 '비트코인 1만7000달러 돌파...,전자신문,https://n.news.naver.com/mnews/article/030/000...,2020-12-02 01:43:33,2020-12-02
1,2020-12-02,2020-12-02 10:41:29,"‘위험자산 GO, 안전자산 STOP’","\r\n주가·비트코인은 사상 최고…금·달러는 약세코스피, 장중 2675.28…사상 ...",헤럴드경제,https://n.news.naver.com/mnews/article/016/000...,2020-12-02 01:41:29,2020-12-02
2,2020-12-02,2020-12-02 10:10:25,<모두가 통하는 쉬운 우리말 생활>포스팅·필터링·스트리밍… 번역어 있는데...,\r\n\r\n\r\n\r\n\r\n(18) 컴퓨터·인터넷 용어컴퓨터와 인터넷 분야...,17면 1단,https://n.news.naver.com/mnews/article/021/000...,2020-12-02 01:10:25,2020-12-02
3,2020-12-02,2020-12-02 12:28:18,카카오 블록체인 계열사서 개인정보 유출··· 이름 이메일 전화번호 포함,\r\n\r\n\r\n\r\n\r\n[디지털데일리 이종현기자] 카카오의 블록체인 자...,디지털데일리,https://n.news.naver.com/mnews/article/138/000...,2020-12-02 03:28:18,2020-12-02
4,2020-12-02,2020-12-02 10:11:14,네이버 공공 클라우드에 ‘람다256 블록체인’ 올라간다,\r\n\t\t\t[이데일리 김현아 기자]\r\n\r\n\r\n\r\n네이버의 클라...,이데일리,https://n.news.naver.com/mnews/article/018/000...,2020-12-02 01:11:14,2020-12-02
5,2020-12-02,2020-12-02 12:00:39,"가상화폐, 러시아 중앙 스베르방크서 공식화 선언",\r\n\r\n\r\n\r\n\r\n비트코인 가격이 사상 최고치를 넘긴 가운데 러시...,조세일보,https://n.news.naver.com/mnews/article/123/000...,2020-12-02 03:00:39,2020-12-02
6,2020-12-02,2020-12-02 20:41:32,간이과세 기준 상향하고 '착한 임대인 공제' 기한 늘린다,\r\n가상화폐 소득에는 2022년부터 소득세 부과중기 직장인의 육아복직 세액공제율...,8면 2단,https://n.news.naver.com/mnews/article/011/000...,2020-12-02 11:41:32,2020-12-02
7,2020-12-02,2020-12-02 15:56:49,"리브라 암호화폐, 페이스북과 거리두기…'디엠'으로 개명",\r\n협회 독립성 강조…기존 규제기관과 우호적 관계 형성[아이뉴스24 안희권 기자...,아이뉴스24,https://n.news.naver.com/mnews/article/031/000...,2020-12-02 06:56:49,2020-12-02
8,2020-12-02,2020-12-02 09:00:19,미 국무부 '대북제재 위반 제보' 웹사이트 개설…北 '압박',\r\n자금 세탁·사이버 작전·사치품 수출 등 제보에 포상금 제공\r\n\r\n\r...,뉴스1,https://n.news.naver.com/mnews/article/421/000...,2020-12-02 00:00:19,2020-12-02
9,2020-12-02,2020-12-02 11:23:03,비트겟(Bitget) 카피 트레이드 입주 트레이더 1600명 돌파,\r\n글로벌 진출 본격화…. 한국 시장 진출도\r\n\r\n\r\n\r\n파생상품...,디지털타임스,https://n.news.naver.com/mnews/article/029/000...,2020-12-02 02:23:03,2020-12-02


In [22]:
filtered_data['content'][13]

'\r\n\r\n\r\n\r\n\r\n암호화폐 비트코인을 형상화한 버튼. 2019.4.3.로이터 연합뉴스곽혜진 기자 demian@seoul.co.kr▶ 네이버에서 서울신문 구독하기 클릭! ▶ 세상에 이런 일이 ▶ [연예계 뒷얘기] 클릭!ⓒ 서울신문(www.seoul.co.kr), 무단전재 및 재배포금지\r\n\t\t'

In [23]:
filtered_data.to_csv('ko_sbert_20201202.csv', encoding='utf-8-sig', index=False)